# Adversarial Search: Solving Tic-Tac-Toe with Monte Carlo Tree Search

## Introduction 

Multiplayer games can be implemented as:
1. Nondeterministic actions: The opponent is seen as part of an environment with nondeterministic actions. Non-determinism is the result of the unknown opponent's moves. 
2. Optimal Decisions: Minimax search (search complete game tree) and alpha-beta pruning.
3. Heuristic Alpha-Beta Tree Search: Cut off tree search and use heuristic to estimate state value. 
4. __Monte Carlo Tree search:__ Simulate playouts to estimate state value. 

Here we will implement search for Tic-Tac-Toe (see [rules](https://en.wikipedia.org/wiki/Tic-tac-toe)). The game is a __zero-sum game__: Win by x results in +1, win by o in -1 and a tie has a value of 0. Max plays x and tries to maximize the outcome while Min plays o and tries to minimize the outcome.   

We will implement
* Pure Monte Carlo search

## State Space and Search Tree Size

Each state is a possible board. Each of the 9 squares can have 3 values (empty, x and o), but some boards are impossible (where a player has several sequences of 3).The number of states in the state space graph is less than:

In [1]:
3**9

19683

A search tree (called game tree) can be superimposed on the state space graph. Note that a state can be in several branches of the tree resulting in more notes. 

* The complete search tree has a maximal depth $m=9$.
* The max branching factor $b=9$ (for first move).

DFS has a time complecity of $O(b^m)$ and a space complexity of $O(bm)$.

In [2]:
9**9

387420489

The search tree has $9 \times 8 \times \dots \times 2 \times 1$ many terminal nodes. This is equivalent to the number of permutations of the sequence 1 through 9:

In [3]:
import math

math.factorial(9)

362880

The number of terminal nodes of the complete search tree are less than (some are not possible because a player won twice, etc. We can reach a complete board with that many sequences. Some sequences are cut short because of a win and therefore there are less terminal notes.

It takes 9 moves to get to a complete board. An upper bound on the tree sice is the number of terminal notes $\times$ max tree depth:

In [4]:
math.factorial(9) * 9

3265920

__Note:__ This size makes this a very small problem that can be easily solved by searching the complete game tree. Most games and real problems are to large and can only be.

## The board

I represent the board as a vector of length 9. The values are `' ', 'x', 'o'`.  

In [5]:
def empty_board():
    return [' '] * 9

board = empty_board()
display(board)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Some helper functions.

In [6]:
import numpy as np

def show_board(board):
    """display the board"""
    b = np.array(board).reshape((3,3))
    print(b)

board = empty_board()
show_board(board)    

print()
print("Add some x's")
board[0] = 'x'; board[3] = 'x'; board[6] = 'x';  
show_board(board)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Add some x's
[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


In [7]:
def check_win(board):
    """check the board and return one of x, o, d (draw), or n (for next move)"""
    
    board = np.array(board).reshape((3,3))
    
    diagonals = np.array([[board[i][i] for i in range(len(board))], 
                          [board[i][len(board)-i-1] for i in range(len(board))]])
    
    for a_board in [board, np.transpose(board), diagonals]:
        for row in a_board:
            if len(set(row)) == 1 and row[0] != ' ':
                return row[0]
    
    # check for draw
    if(np.sum(board == ' ') < 1):
        return 'd'
    
    return 'n'

show_board(board)
print('Win? ' + check_win(board))

print()
show_board(empty_board())
print('Win? ' + check_win(empty_board()))

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]
Win? x

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
Win? n


In [8]:
def get_actions(board):
    """return possible actions as a vector ot indices"""
    return np.where(np.array(board) == ' ')[0].tolist()

    # randomize the action order
    #actions = np.where(np.array(board) == ' ')[0]
    #np.random.shuffle(actions)
    #return actions.tolist()


show_board(board)
get_actions(board)

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


[1, 2, 4, 5, 7, 8]

In [9]:
def result(state, player, action):
    """Add move to the board."""
    
    state = state.copy()
    state[action] = player
  
    return state

show_board(empty_board())

print()
print("State for placing an x at position 4:")
show_board(result(empty_board(), 'x', 4))

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

State for placing an x at position 4:
[[' ' ' ' ' ']
 [' ' 'x' ' ']
 [' ' ' ' ' ']]


In [10]:
def utility(state):
    """check is a state is terminal and return the utility if it is. None means not a terminal mode."""
    goal = check_win(state)        
    if goal == 'x': return +1 
    if goal == 'd': return 0  
    if goal == 'o': return -1  # loss is failure
    return None # continue

print(utility(['x'] * 9))
print(utility(['o'] * 9))
print(utility(empty_board()))

1
-1
None


# Pure Monte Carlo Tree Search

See AIMA page 161ff. 

We implement a extremely simplified version.

For the current state: 
1. Simulate $N$ random playouts for each possible action and 
2. pick the action with the highest average utility.

__Important note:__ we use here a random playout policy, which ends up creating just a randomized search that works fine for this toy problem. For real applications you need to extend the code with
1. a good __playout policy__ (e.g., learned by self-play) and 
2. a __selection policy__ (e.g., UCB1).

## Simulate playouts

In [11]:
def playout(state, action):
    state = result(state, 'x', action)
    player = 'o'
    
    while(True):
        u = utility(state)
        if u is not None: return(u)
        
        # we use a random playout policy
        a = np.random.choice(get_actions(state))
        state = result(state, player, a)
        #print(state)
        
        if player == 'o': 
            player = 'x'
        else: 
            player = 'o'


board = empty_board()
print(playout(board, 0))
print(playout(board, 0))
print(playout(board, 0))

1
1
-1


In [12]:
def rep_playout(board, action, N = 100):
    return [playout(board, action) for i in range(N)]

p = rep_playout(board, 0)
print(p)

print(f"mean utility: {np.mean(p)}")
print(f"win probability: {(np.mean(p) + 1)/2}")

[0, 1, -1, 0, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 0, 1, -1, 1, 0, 1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 0, 0, -1, 1, 1, 1, 1, -1, 1, 0, 0, 1, 1, 0, 1, -1, 1, 1, -1, 1, 0, 0, -1, -1, 1, -1, 1, 1, 1, -1, 0, 1, 1, 1, 1, -1, 1, 1, 1, 1, 0, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1, 1, 1, 1, 1, 0]
mean utility: 0.37
win probability: 0.685


__Note:__ This shows that the player who goes first has a significant advantage in pure random play.

## Choose the best action

Pure Monte Carlo Search (pmcs)

In [13]:
def pmcs(board, N = 100, debug = False):
    ps = [[i, np.mean(rep_playout(board, i, N = N))] for i in get_actions(board)]

    if debug: display(ps)
        
    action = sorted(ps, key=lambda ps: ps[1], reverse=True)[0][0]
    return action

pmcs(board, debug = True)

[[0, 0.47],
 [1, 0.07],
 [2, 0.46],
 [3, 0.24],
 [4, 0.61],
 [5, 0.18],
 [6, 0.48],
 [7, 0.37],
 [8, 0.33]]

4

Looks like the center and the corners are a lot better.

## Some Tests

In [14]:
# x is about to win (play 8)

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'

print("Board:")
show_board(board)

print()
display(pmcs(board, debug = True))

Board:
[['x' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' ' ']]



[[2, 0.88], [5, 0.67], [6, 0.83], [7, 0.76], [8, 1.0]]

8

In [15]:
# o is about to win

board = empty_board() 
board[0] = 'o'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'
board[8] = 'x'

print("Board:")
show_board(board)

print()
display(pmcs(board, debug = True))

Board:
[['o' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' 'x']]



[[2, 0.12], [5, -0.62], [6, 0.02], [7, -0.62]]

2

In [16]:
#### x can draw if it chooses 7.

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[2] = 'x'
board[4] = 'o'

print("Board:")
show_board(board)

print()
display(pmcs(board, debug = True))

Board:
[['x' 'o' 'x']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]



[[3, -0.15], [5, -0.28], [6, -0.27], [7, 0.06], [8, -0.34]]

7

In [17]:
# o went first

board = empty_board() 
board[4] = 'o'

print("Board:")
show_board(board)


print()
display(pmcs(board, debug = True))

Board:
[[' ' ' ' ' ']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]



[[0, -0.44],
 [1, -0.63],
 [2, -0.37],
 [3, -0.69],
 [5, -0.5],
 [6, -0.46],
 [7, -0.53],
 [8, -0.44]]

2

In [18]:
# Empty board: Only a draw an be guaranteed

board = empty_board() 

print("Board:")
show_board(board)


print()
display(pmcs(board, debug = True))

Board:
[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]



[[0, 0.4],
 [1, 0.41],
 [2, 0.24],
 [3, 0.15],
 [4, 0.6],
 [5, 0.25],
 [6, 0.3],
 [7, 0.16],
 [8, 0.27]]

4

In [19]:
# A bad situation

board = empty_board() 
board[0] = 'o'
board[2] = 'x'
board[8] = 'o'

print("Board:")
show_board(board)


print()
display(pmcs(board, debug = True))

Board:
[['o' ' ' 'x']
 [' ' ' ' ' ']
 [' ' ' ' 'o']]



[[1, -0.72], [3, -0.59], [4, 0.2], [5, -0.58], [6, -0.14], [7, -0.68]]

4